# Data Mining Project - Group XX 2025/2026

# Import Libraries

In [1]:
import sqlite3
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil

from itertools import product
from ydata_profiling import ProfileReport

# for better resolution plots
%config InlineBackend.figure_format = 'retina'

#o svg consegue ampliar infinitamente os gráficos sem perder qualidade mas às vezes é mais lento 
#por isso agora usamos retina


sns.set()

# Loading the Data

Import the datasets from csv files using commas as separators of the columns and setting the unique customer identifier as the index of both columns.

In [2]:
flightsDB = pd.read_csv('data/DM_AIAI_FlightsDB.csv', sep = ",", index_col= "Loyalty#")
customerDB = pd.read_csv('data/DM_AIAI_CustomerDB.csv', sep = ",", index_col= "Loyalty#")
metaData = pd.read_csv('data/DM_AIAI_Metadata.csv', sep = ";", header= None)

Remove the 'Unnamed' column referring to a sequential numbering of the rows, as we set the column "Loyalty#" as the index

In [3]:
customerDB = customerDB.iloc[:, 1:]
customerDB

,First Name,Last Name,Customer Name,Country,Province or State,City,Latitude,Longitude,Postal code,Gender,Education,Location Code,Income,Marital Status,LoyaltyStatus,EnrollmentDateOpening,CancellationDate,Customer Lifetime Value,EnrollmentType
Loyalty#,,,,,,,,,,,,,,,,,,,
480934,Cecilia,Householder,Cecilia Householder,Canada,Ontario,Toronto,43.653225,-79.383186,M2Z 4K1,female,Bachelor,Urban,70146.0,Married,Star,2/15/2019,NaN,3839.14,Standard
549612,Dayle,Menez,Dayle Menez,Canada,Alberta,Edmonton,53.544388,-113.490930,T3G 6Y6,male,College,Rural,0.0,Divorced,Star,3/9/2019,NaN,3839.61,Standard
429460,Necole,Hannon,Necole Hannon,Canada,British Columbia,Vancouver,49.282730,-123.120740,V6E 3D9,male,College,Urban,0.0,Single,Star,7/14/2017,1/8/2021,3839.75,Standard
608370,Queen,Hagee,Queen Hagee,Canada,Ontario,Toronto,43.653225,-79.383186,P1W 1K4,male,College,Suburban,0.0,Single,Star,2/17/2016,NaN,3839.75,Standard
530508,Claire,Latting,Claire Latting,Canada,Quebec,Hull,45.428730,-75.713364,J8Y 3Z5,male,Bachelor,Suburban,97832.0,Married,Star,10/25/2017,NaN,3842.79,2021 Promotion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100012,Ethan,Thompson,Ethan Thompson,Canada,Quebec,Quebec City,46.759733,-71.141009,Y0C 7D6,male,Bachelor,Suburban,NaN,Single,Star,2/27/2019,2/27/2019,NaN,Standard
100013,Layla,Young,Layla Young,Canada,Alberta,Edmonton,53.524829,-113.546357,L3S 9Y3,female,Bachelor,Rural,NaN,Married,Star,9/20/2017,9/20/2017,NaN,Standard
100014,Amelia,Bennett,Amelia Bennett,Canada,New Brunswick,Moncton,46.051866,-64.825428,G2S 2B6,male,Bachelor,Rural,NaN,Married,Star,11/28/2020,11/28/2020,NaN,Standard


# Metadata

**FlightsDB Database Variable Description**
- **Loyalty#:**	Unique customer identifier linking to CustomerDB
- **Year:**	Year of flight activity record
- **Month:**	Month of flight activity record (1-12)
- **YearMonthDate:**	First day of the month for the activity period
- **NumFlights:**	Total number of flights taken by customer in the month
- **NumFlightsWithCompanions:**	Number of flights where customer traveled with companions
- **DistanceKM:**	Total distance traveled in kilometers for the month
- **PointsAccumulated:**	Loyalty points earned by customer during the month
- **PointsRedeemed:**	Loyalty points spent/redeemed by customer during the month
- **DollarCostPointsRedeemed:**	Dollar value of points redeemed during the month

**CustomerDB Database Variable Description**
- **Loyalty#:**  Unique customer identifier for loyalty program members
- **First Name:**   Customer's first name
- **Last Name:**   Customer's last name 
- **Customer Name:** Customer's full name (concatenated)
- **Country:**	Customer's country of residence
- **Province or State:**	Customer's province or state
- **City:**	Customer's city of residence
- **Latitude:**	Geographic latitude coordinate of customer location
- **Longitude:**	Geographic longitude coordinate of customer locatio
- **Postal code:**	Customer's postal/ZIP code
- **Gender:**	Customer's gender
- **Education:**	Customer's highest education level (Bachelor, College, etc.)
- **Location:** Code	Urban/Suburban/Rural classification of customer residence
- **Income:**	Customer's annual income
- **Marital Status:**	Customer's marital status (Married, Single, Divorced)
- **LoyaltyStatus:**	Current tier status in loyalty program (Star > Nova > Aurora)
- **EnrollmentDateOpening:**	Date when customer joined the loyalty program
- **CancellationDate:**	Date when customer left the program
- **Customer Lifetime:** Value	Total calculated monetary value of customer relationship
- **EnrollmentType:**	Method of joining loyalty program

# Data Preparation

## Feature Engineering (FlightsDB)

Firstly, we have decided to add the following features aggregated by Loyalty# so that we have the totals of the existing features. The intent of this is to help us create even more meaningful features next and give the full picture of the existing monthly data.
- `TotalFlights:` Total number of flights taken by the customer over their entire membership period.

- `TotalFlightsWithCompanions:` Total number of flights taken with at least one companion.

- `TotalDistanceKM:` Total distance flown (in kilometers) across all flights taken by the customer.

- `TotalPointsAccumulated:` Total loyalty points earned throughout the customer’s history.

- `TotalPointsRedeemed:` Total loyalty points redeemed by the customer.

- `TotalAvgDistancePerFlight:` Average distance per flight calculated across all trips in the customer’s history.


In [4]:
# Convert YearMonthDate to datetime
flightsDB['YearMonthDate'] = pd.to_datetime(flightsDB['YearMonthDate'])
flightsDB['Month'] = flightsDB['YearMonthDate'].dt.month
flightsDB['Year'] = flightsDB['YearMonthDate'].dt.year

# Aggregate by customer (lifetime summary metrics)
flights_agg = flightsDB.groupby('Loyalty#').agg({
    'NumFlights': 'sum',
    'NumFlightsWithCompanions': 'sum',
    'DistanceKM': 'sum',
    'PointsAccumulated': 'sum',
    'PointsRedeemed': 'sum',
}).reset_index()

# Rename aggregated columns directly
flights_agg.rename(columns={
    'NumFlights': 'TotalFlights',
    'NumFlightsWithCompanions': 'TotalFlightsWithCompanions',
    'DistanceKM': 'TotalDistanceKM',
    'PointsAccumulated': 'TotalPointsAccumulated',
    'PointsRedeemed': 'TotalPointsRedeemed',
}, inplace=True)

# Add new aggregated features to the flightsDB
flightsDB_expanded = flightsDB.merge(flights_agg, on='Loyalty#', how='left')

# Preview
flightsDB_expanded.describe().T


,count,mean,min,25%,50%,75%,max,std
Loyalty#,608436.0,550037.873084,100018.0,326961.0,550834.0,772194.0,999986.0,258935.180575
Year,608436.0,2020.0,2019.0,2019.0,2020.0,2021.0,2021.0,0.816497
Month,608436.0,6.5,1.0,3.75,6.5,9.25,12.0,3.452055
YearMonthDate,608436,2020-06-16 02:40:00,2019-01-01 00:00:00,2019-09-23 12:00:00,2020-06-16 00:00:00,2021-03-08 18:00:00,2021-12-01 00:00:00,NaN
NumFlights,608436.0,3.908107,0.0,0.0,0.0,7.2,21.0,5.057889
NumFlightsWithCompanions,608436.0,0.983944,0.0,0.0,0.0,0.9,11.0,2.003785
DistanceKM,608436.0,7939.341419,0.0,0.0,856.4,15338.175,42040.0,10260.421873
PointsAccumulated,608436.0,793.777781,0.0,0.0,85.275,1533.7125,4204.0,1025.918521
PointsRedeemed,608436.0,235.251678,0.0,0.0,0.0,0.0,7496.0,983.233374
DollarCostPointsRedeemed,608436.0,2.324835,0.0,0.0,0.0,0.0,74.0,9.725168


We will add the following features to our model in order to better analyse and segment the customers in the future.

- `PointsRedemptionRatio:` The proportion of earned points that a customer has redeemed. Calculated as PointsRedeemed / PointsAccumulated. Shows engagement with the loyalty program and redemption behavior.

- `FlightWithCompanionRatio:` Ratio of flights where the customer traveled with companions. Calculated as NumFlightsWithCompanions / NumFlights. Reveals travel habits for group or family targeting.

- `FlightsVariance:` Variance or standard deviation of number of flights per customer over time. Calculated as groupby(Loyalty#)['NumFlights'].var(). Measures consistency in travel patterns.

- `Recency:` Number of months since the customer last took a trip. Calculated as the time difference between the dataset’s latest recorded date and the customer’s most recent active flight month. A core RFM metric used to evaluate engagement freshness and identify churn risk.

- `DistancePerFlight`: The average distance flown per flight per customer. This allows us to see which customers make short and long trips.

In [ ]:
# Latest date in dataset 
latest_date = flightsDB['YearMonthDate'].max()

# Group by customer using aggregated values
grouped = flightsDB_expanded.groupby('Loyalty#')

# PointsRedemptionRatio (Total)
flightsDB_expanded['PointsRedemptionRatio'] = np.where(
    flightsDB_expanded['TotalPointsAccumulated'] > 0,
    flightsDB_expanded['TotalPointsRedeemed'] / flightsDB_expanded['TotalPointsAccumulated'],
    0
)

# Total Flight With Companion Ratio (lifetime)
flightsDB_expanded['TotalFlightWithCompanionRatio'] = np.where(
    flightsDB_expanded['TotalFlights'] > 0,
    flightsDB_expanded['TotalFlightsWithCompanions'] / flightsDB_expanded['TotalFlights'],
    0
)

# FlightsVariance (month-to-month consistency)
flightsDB_expanded['FlightsVariance'] = grouped['NumFlights'].transform('var').fillna(0)

# Recency (months since last active flight)
active_flights = flightsDB_expanded[flightsDB_expanded['NumFlights'] > 0]
last_active_flight = active_flights.groupby('Loyalty#')['YearMonthDate'].max()

flightsDB_expanded['Recency'] = (
    (latest_date - flightsDB_expanded['Loyalty#'].map(last_active_flight)).dt.days / 30
).fillna(999).astype(int)

# Total Distance per Flight
flightsDB_expanded['DistancePerFlight'] = np.where(
    flightsDB_expanded['TotalFlights'] > 0,
    flightsDB_expanded['TotalDistanceKM'] / flightsDB_expanded['TotalFlights'],
    0
)

flightsDB_expanded.describe().T

,count,mean,min,25%,50%,75%,max,std
Loyalty#,608436.0,550037.873084,100018.0,326961.0,550834.0,772194.0,999986.0,258935.180575
Year,608436.0,2020.0,2019.0,2019.0,2020.0,2021.0,2021.0,0.816497
Month,608436.0,6.5,1.0,3.75,6.5,9.25,12.0,3.452055
YearMonthDate,608436,2020-06-16 02:40:00,2019-01-01 00:00:00,2019-09-23 12:00:00,2020-06-16 00:00:00,2021-03-08 18:00:00,2021-12-01 00:00:00,NaN
NumFlights,608436.0,3.908107,0.0,0.0,0.0,7.2,21.0,5.057889
NumFlightsWithCompanions,608436.0,0.983944,0.0,0.0,0.0,0.9,11.0,2.003785
DistanceKM,608436.0,7939.341419,0.0,0.0,856.4,15338.175,42040.0,10260.421873
PointsAccumulated,608436.0,793.777781,0.0,0.0,85.275,1533.7125,4204.0,1025.918521
PointsRedeemed,608436.0,235.251678,0.0,0.0,0.0,0.0,7496.0,983.233374
DollarCostPointsRedeemed,608436.0,2.324835,0.0,0.0,0.0,0.0,74.0,9.725168


## Feature Engineering (CustomerDB)

We have decided to add the following features to help us segment the customers later on.
- `IncomeBins:` Groups customers into discrete income categories ranging from low to very high.

- `HouseholdType:` Combined indicator describing both marital status and location type of the customer.

- `MonthsSinceEnrollment:` Number of months since the customer joined the loyalty program.

- `IsActive:` Indicates whether the customer is currently active in the loyalty program (no cancellation date).

- `CLVBins:` Categorizes customers into five groups based on their Customer Lifetime Value (CLV), from lowest to highest.

- `IncomeVsEducationMean`: Difference between the customer’s income and the average income for their education group.

- `CLVvsCityMean`: Difference between the customer’s CLV and the average CLV for their city.


In [ ]:
# Create expanded dataset
customerDB_expanded = customerDB.copy()

# Convert enrollment and cancellation dates to datetime
customerDB['EnrollmentDateOpening'] = pd.to_datetime(customerDB_expanded['EnrollmentDateOpening'], errors='coerce')

# Today reference (or max date from flightsDB if preferred)
reference_date = pd.Timestamp.today()

## INCOME BINS
# Fill NaNs temporarily with -1
income_nonan = customerDB_expanded['Income'].fillna(-1)

# Compute quartiles
try:
    temp_bins = pd.qcut(income_nonan, q=4, duplicates='drop')
    bins_labels = ['Very Low', 'Low', 'Medium', 'High', 'Very High'][:len(temp_bins.cat.categories)]
    customerDB_expanded['IncomeBins'] = pd.qcut(
        income_nonan,
        q=4,
        labels=bins_labels,
        duplicates='drop'
    )
    # Replace the -1 bin with 'Unknown'
    customerDB_expanded['IncomeBins'] = customerDB_expanded['IncomeBins'].cat.add_categories(['Unknown'])
    customerDB_expanded.loc[income_nonan == -1, 'IncomeBins'] = 'Unknown'
except ValueError:
    customerDB_expanded['IncomeBins'] = pd.Series(['Unknown']*len(customerDB_expanded))



# HouseholdType → Marital + Location
customerDB_expanded['HouseholdType'] = (
    customerDB_expanded['Location Code'].fillna('Unknown') + '-' +
    customerDB_expanded['Marital Status'].fillna('Unknown')
)

# MonthsSinceEnrollment
customerDB_expanded['MonthsSinceEnrollment'] = (
    (reference_date - customerDB['EnrollmentDateOpening']).dt.days / 30
).fillna(0).clip(lower=0)

# IsActive → No cancellation = still active
customerDB_expanded['IsActive'] = customerDB_expanded['CancellationDate'].isna().astype(int)

# Fill NaNs temporarily with -1
clv_nonan = customerDB_expanded['Customer Lifetime Value'].fillna(-1)

## CLV BINS
# Compute quartiles
try:
    temp_bins = pd.qcut(clv_nonan, q=5, duplicates='drop')
    clv_labels = ["Very Low", "Low", "Medium", "High", "Very High"][:len(temp_bins.cat.categories)]
    customerDB_expanded['CLVBins'] = pd.qcut(
        clv_nonan,
        q=5,
        labels=clv_labels,
        duplicates='drop'
    )
    # Replace the -1 bin with 'Unknown'
    customerDB_expanded['CLVBins'] = customerDB_expanded['CLVBins'].cat.add_categories(['Unknown'])
    customerDB_expanded.loc[clv_nonan == -1, 'CLVBins'] = 'Unknown'
except ValueError:
    customerDB_expanded['CLVBins'] = pd.Series(['Unknown']*len(customerDB_expanded))

# ---  Income vs Education Mean ---
customerDB_expanded['IncomeVsEducationMean'] = (
    customerDB_expanded['Income'] - customerDB_expanded.groupby('Education')['Income'] \
    .transform('mean')
)

# ---  CLV vs City Mean ---
customerDB_expanded['CLVvsCityMean'] = (
    customerDB_expanded['Customer Lifetime Value'] - customerDB_expanded.groupby('City')['Customer Lifetime Value'] \
    .transform('mean')
)

# Preview result
customerDB_expanded.describe(include='all').T


,count,unique,top,freq,mean,min,25%,50%,75%,max,std
First Name,16921,4941,Deon,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Last Name,16921,15404,Salberg,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Customer Name,16921,16921,Cecilia Householder,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Country,16921,1,Canada,16921,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Province or State,16921,11,Ontario,5468,NaN,NaN,NaN,NaN,NaN,NaN,NaN
City,16921,29,Toronto,3390,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Latitude,16921.0,NaN,NaN,NaN,47.1745,42.984924,44.231171,46.087818,49.28273,60.721188,3.307971
Longitude,16921.0,NaN,NaN,NaN,-91.814768,-135.05684,-120.23766,-79.383186,-74.596184,-52.712578,22.242429
Postal code,16921,75,V6E 3D9,917,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Gender,16921,2,female,8497,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Feature Engineering with Merged Datasets

### Merge Datasets

In [9]:
# Create a second expanded flightsDB, keeping only one row per customer with aggregated features
flightsDB_expanded2 = flightsDB.merge(flights_agg, on='Loyalty#', how='right')

# Merge customerDB_expanded with flightsDB_expanded2 on Loyalty#
dfs_merged = customerDB_expanded.join(flightsDB_expanded2, on="Loyalty#", how="left")

# Fill only numeric columns with 0
num_cols = dfs_merged.select_dtypes(include=['number']).columns
dfs_merged[num_cols] = dfs_merged[num_cols].fillna(0)

# Keep categorical NA values untouched

### Add Features

We have decided to add the following features to help us segment the customers later on.
- `FlightsPerMonthOfMembership:` Average number of flights a customer takes per month of membership. Calculated as TotalFlights / MonthsSinceEnrollment. Captures travel frequency relative to how long the customer has been in the program.

- `DistancePerCLV:` Total distance traveled by the customer normalized by their Customer Lifetime Value (CLV). Calculated as TotalDistanceKM / Customer Lifetime Value. Helps identify high-travel customers relative to their monetary value.

- `PointsRedeemedPerTotalCLV:` Loyalty points redeemed per dollar of CLV. Calculated as TotalPointsRedeemed / Customer Lifetime Value. Shows engagement efficiency in terms of spending vs. earned value.

- `RedemptionRateByLoyaltyStatus:` Customer’s points redemption relative to the average points redeemed for their loyalty tier. Calculated as TotalPointsRedeemed / mean(TotalPointsRedeemed per LoyaltyStatus). Captures how actively customers in a tier redeem points compared to peers.

- `DistanceTraveledByEducationLevel:` Customer’s total distance traveled relative to the average distance for their education level. Calculated as TotalDistanceKM / mean(TotalDistanceKM per Education group). Helps understand travel behavior patterns by education segment.


In [13]:
# Defensive helper for zero-division
def safe_div(numerator, denominator):
    return (numerator / denominator).replace([np.inf, -np.inf], 0).fillna(0)


# FlightsPerMonthOfMembership
dfs_merged["FlightsPerYearOfMembership"] = safe_div(
    dfs_merged["TotalFlights"],
    dfs_merged["MonthsSinceEnrollment"]
)

# DistancePerCLV
dfs_merged["DistancePerCLV"] = safe_div(
    dfs_merged["TotalDistanceKM"],
    dfs_merged["Customer Lifetime Value"]
)

# PointsRedeemedPerCLVDollar
dollar_redeemed = flightsDB.groupby('Loyalty#')['DollarCostPointsRedeemed'].sum()
dfs_merged["PointsRedeemedPerCLVDollar"] = safe_div(
    dfs_merged["TotalPointsRedeemed"],
    dfs_merged["Loyalty#"].map(dollar_redeemed)
)

# RedemptionRateByLoyaltyStatus
loyalty_points_mean = dfs_merged.groupby("LoyaltyStatus")["TotalPointsRedeemed"].transform("mean")
dfs_merged["RedemptionRateByLoyaltyStatus"] = safe_div(
    dfs_merged["TotalPointsRedeemed"],
    loyalty_points_mean
)

# DistanceTraveledByEducationLevel
edu_distance_mean = dfs_merged.groupby("Education")["TotalDistanceKM"].transform("mean")
dfs_merged["DistanceTraveledByEducationLevel"] = safe_div(
    dfs_merged["TotalDistanceKM"],
    edu_distance_mean
)


# Final result preview
dfs_merged.describe().T


,count,mean,min,25%,50%,75%,max,std
Latitude,16921.0,47.1745,42.984924,44.231171,46.087818,49.28273,60.721188,3.307971
Longitude,16921.0,-91.814768,-135.05684,-120.23766,-79.383186,-74.596184,-52.712578,22.242429
Income,16921.0,37713.409787,0.0,0.0,34112.0,62375.0,99981.0,30378.754775
Customer Lifetime Value,16921.0,7981.015757,0.0,3971.35,5771.61,8936.82,83325.38,6864.608367
MonthsSinceEnrollment,16921.0,86.153007,46.766667,64.733333,85.3,107.033333,131.066667,23.965966
IsActive,16921.0,0.863483,0.0,1.0,1.0,1.0,1.0,0.343347
IncomeVsEducationMean,16921.0,-0.0,-36388.523445,-9123.523445,0.0,7286.476555,43583.476555,17519.302264
CLVvsCityMean,16921.0,0.0,-6767.170102,-3996.415625,-2142.902095,962.220831,75236.176746,6853.83106
Loyalty#,16921.0,550197.393771,100011.0,326823.0,550896.0,772438.0,999999.0,259251.503597
Year,16921.0,1141.979138,0.0,0.0,2019.0,2020.0,2021.0,1001.376131


In [14]:
len(dfs_merged.columns)

46